In [ ]:
import numpy as np

In [ ]:
# The following routines are for 32-bits
# TODO: have onese for 8, 16, and 64 bits

def part1by1(n):
    n&= 0x0000ffff
    n = (n | (n << 8)) & 0x00FF00FF
    n = (n | (n << 4)) & 0x0F0F0F0F
    n = (n | (n << 2)) & 0x33333333
    n = (n | (n << 1)) & 0x55555555
    return n

def unpart1by1(n):
    n&= 0x55555555
    n = (n ^ (n >> 1)) & 0x33333333
    n = (n ^ (n >> 2)) & 0x0f0f0f0f
    n = (n ^ (n >> 4)) & 0x00ff00ff
    n = (n ^ (n >> 8)) & 0x0000ffff
    return n

def interleave2(x, y):
    return part1by1(x) | (part1by1(y) << 1)

def deinterleave2(n):
    return unpart1by1(n), unpart1by1(n >> 1)

In [ ]:
2**10-1 & 0x000003ff

1023

In [ ]:
# The following routines are for 32-bits
# TODO: have onese for 8, 16, and 64 bits

def part1by2(n):
    n&= 0x000003ff
    n = (n ^ (n << 16)) & 0xff0000ff
    n = (n ^ (n <<  8)) & 0x0300f00f
    n = (n ^ (n <<  4)) & 0x030c30c3
    n = (n ^ (n <<  2)) & 0x09249249
    return n


def unpart1by2(n):
    n&= 0x09249249
    n = (n ^ (n >>  2)) & 0x030c30c3
    n = (n ^ (n >>  4)) & 0x0300f00f
    n = (n ^ (n >>  8)) & 0xff0000ff
    n = (n ^ (n >> 16)) & 0x000003ff
    return n

def interleave3(x, y, z):
    return part1by2(x) | (part1by2(y) << 1) | (part1by2(z) << 2)

def deinterleave3(n):
    return unpart1by2(n), unpart1by2(n >> 1), unpart1by2(n >> 2)

# interleaving in 3D works for x,y,z  $\in 0 \ldots 2^{10}-1 = 0 \ldots 1023$ voxels

... in every dimensions, i.e., only 10-10-10 bits can be used, 10-11-11 does not

It can be reprogrammed that bits 11 are used for 2 coordinates (scheme 11-11-10)


In [ ]:
for x in 8,9,10:
    for y in 8,9,10:
        for z in 8,9,10:
            if deinterleave3 (interleave3(2**x-1,2**y-1,2**z-1)) != (2**x-1,2**y-1,2**z-1):
                print (x,y,z)
            

# Conversion between darts and tuples

In 2D we had binary encoding
```
    xy.xy.xy.xy -- a - bb
```

Tn 3D it translates to
```
    xyz.xyz.xyz.xyz -- aa - bbbb

```

This means:

- x,y,z coordinates of the reference voxel
- 2 bits `aa` to encode the axis (0,1,2) or (R,G,B)
- 4 bits `bbbb` to one of the 16 darts of the sandwich. This can be read as either 
    - index $i \in \{ 0 \ldots 15\}$
    - side of the sandwich $s \in \{ 0, 1\}$  plus  index within the side $i \in \{ 0 \ldots 7\}$
    
Conversion routines follow:

In [ ]:
def t5dart (x,y,z,a,i):
    """
    5-tuple to dart.

    x,y,z, voxel coordinates
    a in 0,1,2 axis, i.e the square (and also dart within it) is orthogonal to this axis
    i in 0..15
    """

    return interleave3 (x,y,z) << 6 | a << 4 | i

def t6dart (x,y,z,a,s,i):
    """
    6-tuple to dart

    x,y,z, voxel coordinates

    a in 0,1,2 axis, i.e the square is orthogonal this axis
    s in 0...1 which side of the sandwich
    i in 0...7 dart number within the square
    """

    return interleave3 (x,y,z) << 6 | a << 4 | s << 3 | i

def dart5t (d):
    """Dart to 5-tuple"""
    i = d & 0b1111; d >>= 4
    a = d & 0b11  ; d >>= 2
    x,y,z = deinterleave3 (d)
    return x,y,z,a,i

def dart6t (d):
    """Dart to 6-tuple"""

    i = d & 0b111; d >>= 3
    s = d & 0b1  ; d >>= 1
    a = d & 0b11 ; d >>= 2
    x,y,z = deinterleave3 (d)
    return x,y,z,a,s,i

In [ ]:
# check it
dart6t (t6dart (511,511,255,1,1,7))

(511, 511, 255, 1, 1, 7)

# Support routines

In [ ]:
import termcolor

def bitcolor(*args, return_str=False, nBits=32, sep=' '):
    'convert number or tuple to colorfull bitcode text'
    colors = 'red green blue'.split()

    if len (args) == 1: n = args[0]
    if len (args) == 2: n = interleave2 (args[0],args[1])
    if len (args) == 3: n = interleave3 (args[0],args[1],args[2])
        
    text = f'{n:0{nBits}b}'
    out = ''
    D = len (args)
    for i,c in enumerate(text,D-len(text)%D):
        out += termcolor.colored(c,colors[i%D])
        if i%D==D-1 and i< len(text): out+=sep
    if return_str:
        return out
    else:
        print (out)

In [ ]:
bitcolor(2**14,2**15)

10 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00


In [ ]:
bitcolor(3,2,1,nBits=9)

000 011 101


In [ ]:
bitcolor(2**10-1,2**10-1,2**10-1, sep='-')

00-111-111-111-111-111-111-111-111-111-111


In [ ]:
def dartcolor (*args, nBits=9, return_str=False):
    """
    prints or returns colored bit-string representing a dart

    dart can be represented by one of the following:
        1 unsigned integer
        5 tuple x,y,z, ax,    i  ....           i in 0...15
        6 tuple x,y,z, ax, s ,i  .... s in 0,1  i in 0... 7
    keywords:
        nbits:       limit the number of shown bits: default 9 
        return_str:  True to return string, False (default)
    """
    
    if len (args) == 1:
        d = args[0]
    if len (args) == 5:
        d = t5dart (*args)
    if len (args) == 6:
        d = t6dart (*args)
    txt = f'{d:0{3*3+2+4}b}'
             
    x,y,z,a,s,i = dart6t (d)
             
    out  = bitcolor(x,y,z,return_str=True, sep='.', nBits=nBits)

    axcolor = 'red green blue'.split()[ a ]
    out += "-" + termcolor.colored (f'{txt[-6:-4]}', on_color='on_'+axcolor)

    out += f'-{txt[-4]}-{txt[-3:]}'  #attrs=['underline']
             
    if return_str:
        return out
    else:
        print (out)    

In [ ]:
dartcolor (20,3,20, 2,1,7)
dartcolor (511,511,255,0,0,0)

101.000.101.010.010-10-1-111
11.111.111.111.111.111.111.111.111-00-0-000


### If we want encoding as follows

    zyx-zyx-zyx -- aa -- bbbb

Than we have 26 bits at disposal:
These could encode 8-9-9 bit coordinates, up to $256 \times 512 \times 512$ meaning that

- x can run from 0..511
- y can run from 0..511
- z can run from 0..255

    yx-zyx-zyx-zyx-zyx-zyx-zyx-zyx-zyx--aa--bbbb

## Involutions within 2D membrane

2D membrane is a "sandwich" of two squares sewed by $\alpha_3$.

- darts 0... 7 upper square
- darts 8...15 lower square

In [ ]:
alpha_0 = lambda d: d ^ 0b0001
alpha_1 = lambda d: d ^ 0b0111 ^ (d<<1 & 4)    ^ (d<<2 & 4) 
alpha_3 = lambda d: d ^ 0b1000

In [ ]:
for d in range (0,16,2):
    e = alpha_1(d)
    print (f'{d:2}: {d:04b}  ⟺  {e:2}: {e:04b}')

 0: 0000  ⟺   7: 0111
 2: 0010  ⟺   1: 0001
 4: 0100  ⟺   3: 0011
 6: 0110  ⟺   5: 0101
 8: 1000  ⟺  15: 1111
10: 1010  ⟺   9: 1001
12: 1100  ⟺  11: 1011
14: 1110  ⟺  13: 1101


# Generating Lookup tables

The aim is to generate the following 48-row (for each dart ending) lookup tables for each of the 3 scenarios:

- new bits
- deltas
- mask lookup tables (XOR and AND)

## Lookup for endings

In [ ]:
LUT_D = np.zeros ((48,3), dtype=np.uint8)

In [ ]:
# this defines alpha_2 in the 0,0,0 corner

CORNERS = {
    0b000000 : 0b010111,
    0b010000 : 0b100111,
    0b100000 : 0b000111,
}

In [ ]:
def a(seq,d):
    "Chain alphas, e.g., a ([3,0,2],d) --> alpha_3 ( alpha_0 ( alpha_2 (d))))"
    
    alphas = [alpha_0, alpha_1, CORNERS.get, alpha_3]

    for i in seq:
        d = alphas[i](d)
    return d

In [ ]:
def csd (seq,d): # color sequence on d
    return dartcolor (a(seq,d),return_str=True)

In [ ]:
# sequence for corners

SEQ_CORNERS = [
    [[],      []       ],
    [[0,1],   [1,2,1,3]],
    [[1,0,1], [3]],
    [[0,1,3], [1,2,0,1,3]]
]

In [ ]:
# proof of concept 1-surface cube

from combinatorial.gmaps import nGmap

ARR = np.full ((4,48),-1,dtype=int)

# uncomment one of the following
seq_int = [1,0,1,3] # trailing sequence for interiors
seq_int = []        # trailing sequence for outer shell

for seq_d, seq_e in SEQ_CORNERS:
    for d,e in CORNERS.items():
        ARR [2, a(seq_d,     d)] = a(seq_e + seq_int,    e)
        ARR [2, a(seq_e,     e)] = a(seq_d + seq_int,    d)
        ARR [2, a(seq_d+[0], d)] = a(seq_e + seq_int+[0],e)
        ARR [2, a(seq_e+[0], e)] = a(seq_d + seq_int+[0],d)

for d in range (48):
    ARR [0,d] = alpha_0(d)
    ARR [1,d] = alpha_1(d)
    ARR [3,d] = d # alpha_3(d)


nGmap.from_alpha_array (ARR)

3-gMap of 48 darts:
  # 0-cells: 8
  # 1-cells: 12
  # 2-cells: 6
  # 3-cells: 1
  # ccs    : 1

In [ ]:
# corners
for seq_d, seq_e in SEQ_CORNERS:
    for d,e in CORNERS.items():
        LUT_D [a(seq_d,     d),0] = a(seq_e,    e)
        LUT_D [a(seq_e,     e),0] = a(seq_d    ,d)

        LUT_D [a(seq_d+[0] ,d),0] = a(seq_e+[0],e)
        LUT_D [a(seq_e+[0], e),0] = a(seq_d+[0],d)

        print (csd (seq_d,     d), ' <--> ', csd(seq_e,    e))
        print (csd (seq_d+[0], d), ' <--> ', csd(seq_e+[0],e))
    print()

000.000.000-00-0-000  <-->  000.000.000-01-0-111
000.000.000-00-0-001  <-->  000.000.000-01-0-110
000.000.000-01-0-000  <-->  000.000.000-10-0-111
000.000.000-01-0-001  <-->  000.000.000-10-0-110
000.000.000-10-0-000  <-->  000.000.000-00-0-111
000.000.000-10-0-001  <-->  000.000.000-00-0-110

000.000.000-00-0-010  <-->  000.000.000-10-1-000
000.000.000-00-0-011  <-->  000.000.000-10-1-001
000.000.000-01-0-010  <-->  000.000.000-00-1-000
000.000.000-01-0-011  <-->  000.000.000-00-1-001
000.000.000-10-0-010  <-->  000.000.000-01-1-000
000.000.000-10-0-011  <-->  000.000.000-01-1-001

000.000.000-00-0-101  <-->  000.000.000-01-1-111
000.000.000-00-0-100  <-->  000.000.000-01-1-110
000.000.000-01-0-101  <-->  000.000.000-10-1-111
000.000.000-01-0-100  <-->  000.000.000-10-1-110
000.000.000-10-0-101  <-->  000.000.000-00-1-111
000.000.000-10-0-100  <-->  000.000.000-00-1-110

000.000.000-00-1-010  <-->  000.000.000-10-1-101
000.000.000-00-1-011  <-->  000.000.000-10-1-100
000.000.000-01-1-

In [ ]:
# interiors: append a(1,0,1,3) to the result of the

for seq_d, seq_e in SEQ_CORNERS:
    for d,e in CORNERS.items():
        LUT_D [a(seq_d,d)    ,2] = a(seq_e+[1,0,1,3],   e)
        LUT_D [a(seq_e,e)    ,2] = a(seq_d+[1,0,1,3],   d)
        LUT_D [a(seq_d+[0],d),2] = a(seq_e+[1,0,1,3,0], e)
        LUT_D [a(seq_e+[0],e),2] = a(seq_d+[1,0,1,3,0], d)


        print (csd (seq_d,     d), ' <--> ', csd(seq_e+[1,0,1,3],   e))
        print (csd (seq_d+[0], d), ' <--> ', csd(seq_e+[1,0,1,3,0], e))
    print()

000.000.000-00-0-000  <-->  000.000.000-01-1-010
000.000.000-00-0-001  <-->  000.000.000-01-1-011
000.000.000-01-0-000  <-->  000.000.000-10-1-010
000.000.000-01-0-001  <-->  000.000.000-10-1-011
000.000.000-10-0-000  <-->  000.000.000-00-1-010
000.000.000-10-0-001  <-->  000.000.000-00-1-011

000.000.000-00-0-010  <-->  000.000.000-10-0-101
000.000.000-00-0-011  <-->  000.000.000-10-0-100
000.000.000-01-0-010  <-->  000.000.000-00-0-101
000.000.000-01-0-011  <-->  000.000.000-00-0-100
000.000.000-10-0-010  <-->  000.000.000-01-0-101
000.000.000-10-0-011  <-->  000.000.000-01-0-100

000.000.000-00-0-101  <-->  000.000.000-01-0-010
000.000.000-00-0-100  <-->  000.000.000-01-0-011
000.000.000-01-0-101  <-->  000.000.000-10-0-010
000.000.000-01-0-100  <-->  000.000.000-10-0-011
000.000.000-10-0-101  <-->  000.000.000-00-0-010
000.000.000-10-0-100  <-->  000.000.000-00-0-011

000.000.000-00-1-010  <-->  000.000.000-10-0-000
000.000.000-00-1-011  <-->  000.000.000-10-0-001
000.000.000-01-1-

In [ ]:
# outer faces

SEQ_FACES = [
    [[],      []       ],
    [[0,1],   [1,2,1,3]],
    [[3], [3]],
    [[3,0,1], [1,2,0,1,3]]
]

for seq_d, seq_e in SEQ_FACES:
    for d in 0,16,32:
        LUT_D [a(seq_d,         d),1] = a ( seq_d + [1,0,1],     d)
        LUT_D [a(seq_d+[0],     d),1] = a ( seq_d + [1,0,1,0],   d)

        LUT_D [a(seq_d+[1,0,1],   d),1] = a ( seq_d,               d)
        LUT_D [a(seq_d+[1,0,1,0], d),1] = a ( seq_d+[0],           d)

        print (csd (seq_d,       d), ' <--> ', csd( seq_d + [1,0,1],     d))
        print (csd (seq_d + [0], d), ' <--> ', csd( seq_d + [1,0,1,0],   d))
    print()

000.000.000-00-0-000  <-->  000.000.000-00-0-101
000.000.000-00-0-001  <-->  000.000.000-00-0-100
000.000.000-01-0-000  <-->  000.000.000-01-0-101
000.000.000-01-0-001  <-->  000.000.000-01-0-100
000.000.000-10-0-000  <-->  000.000.000-10-0-101
000.000.000-10-0-001  <-->  000.000.000-10-0-100

000.000.000-00-0-010  <-->  000.000.000-00-0-111
000.000.000-00-0-011  <-->  000.000.000-00-0-110
000.000.000-01-0-010  <-->  000.000.000-01-0-111
000.000.000-01-0-011  <-->  000.000.000-01-0-110
000.000.000-10-0-010  <-->  000.000.000-10-0-111
000.000.000-10-0-011  <-->  000.000.000-10-0-110

000.000.000-00-1-000  <-->  000.000.000-00-1-101
000.000.000-00-1-001  <-->  000.000.000-00-1-100
000.000.000-01-1-000  <-->  000.000.000-01-1-101
000.000.000-01-1-001  <-->  000.000.000-01-1-100
000.000.000-10-1-000  <-->  000.000.000-10-1-101
000.000.000-10-1-001  <-->  000.000.000-10-1-100

000.000.000-00-1-010  <-->  000.000.000-00-1-111
000.000.000-00-1-011  <-->  000.000.000-00-1-110
000.000.000-01-1-

## Lookup for deltas

In [ ]:
# xyz Deltas are too dependent on the CORNERS config, but here it is hard-coded

delta_def = np.fromstring ('''\
  0  0  0     0 -1  0    -1  0  0
  0  0 +1     0  0 +1    -1  0 +1
  0 +1  0     0 +1  0    -1 +1  0
  0  0  0     0  0 -1    -1  0  0
''', sep = ' ', dtype=int).reshape (4,3,3)

In [ ]:
# 4-dimensional index 
# - ax (0,1,2), 
# - d (0-15)
# - scenario (edge, face, interior)
# - delta (z,y,x)

LUT_DELTA = np.zeros ((3,16,3,3), dtype=np.int8)

# red darts alpha_0 replication
LUT_DELTA [0,0:2] = delta_def[0]
LUT_DELTA [0,2:4] = delta_def[1]
LUT_DELTA [0,4:6] = delta_def[2]
LUT_DELTA [0,6:8] = delta_def[3]

# red darts alpha_3 mirroring
LUT_DELTA [0,8:16] = LUT_DELTA[0,0:8,::-1]

# rotation for green and blue darts
LUT_DELTA [1] = np.roll (LUT_DELTA[0], shift=1, axis=-1)
LUT_DELTA [2] = np.roll (LUT_DELTA[1], shift=1, axis=-1)

assert all (LUT_DELTA.sum(axis=(0,1)).flatten() == 0), "sum of deltas for each axis and scenario should be zero"

LUT_DELTA  = LUT_DELTA.reshape  (48,3,3)

## Summary Deltas and 6-bit endings for the 3 scenarios

- 6-bits and Deltas are independent of $S,R,C$
- the only dependency here is how $\alpha_2$ is defined in the 0,0,0 corner
- columns below are 
    - the last 6 bits of dart $d$ in query
    - for each of the 3 scenarios
        - the delta triplets $(\Delta z, \Delta y, \Delta x)$
        - the last 6 bits of $\alpha_2 (d)$

In [ ]:
syms = [
    termcolor.colored('-',color='blue', attrs=['bold']),
#   termcolor.colored('○',color='white'),
    termcolor.colored('◦',color='white'),
    termcolor.colored('+',color='red', attrs=['bold']),
]

print ('  d ending |   270°-flip @ edge  | 180°-flip @ surface | 90°-flip @ interior')
print ('  ax-b-bbb |   z y x  ax-b-bbb   |   z y x  ax-b-bbb   |   z y x  ax-b-bbb')

for d in range (48):
    sd = dartcolor (d, return_str = True)
    scenarios = [dartcolor (dd, return_str=True) for dd in LUT_D[d]]
    
    if not d % 8: print (78*'-')

#   print (f'{d:2} ({sd[-17:]})', end=' -->   ')
#   print (f'[{LUT_DELTA[d,0,0]:2} {LUT_DELTA[d,0,1]:2} {LUT_DELTA[d,0,2]:2}] {LUT_D[d,0]:2} ({scenarios[0][-17:]})', end='   ')
#   print (f'[{LUT_DELTA[d,1,0]:2} {LUT_DELTA[d,1,1]:2} {LUT_DELTA[d,1,2]:2}] {LUT_D[d,1]:2} ({scenarios[1][-17:]})', end='   ')
#   print (f'[{LUT_DELTA[d,2,0]:2} {LUT_DELTA[d,2,1]:2} {LUT_DELTA[d,2,2]:2}] {LUT_D[d,2]:2} ({scenarios[2][-17:]})')


    print (f'  {sd[-17:]} |   ', end='')
    print (f'{syms[LUT_DELTA[d,0,0]+1]} {syms[LUT_DELTA[d,0,1]+1]} {syms[LUT_DELTA[d,0,2]+1]}  {scenarios[0][-17:]}   |   ', end='')
    print (f'{syms[LUT_DELTA[d,1,0]+1]} {syms[LUT_DELTA[d,1,1]+1]} {syms[LUT_DELTA[d,1,2]+1]}  {scenarios[1][-17:]}   |   ', end='')
    print (f'{syms[LUT_DELTA[d,2,0]+1]} {syms[LUT_DELTA[d,2,1]+1]} {syms[LUT_DELTA[d,2,2]+1]}  {scenarios[2][-17:]}   |   ')

print (78*'-')


  d ending |   270°-flip @ edge  | 180°-flip @ surface | 90°-flip @ interior
  ax-b-bbb |   z y x  ax-b-bbb   |   z y x  ax-b-bbb   |   z y x  ax-b-bbb
------------------------------------------------------------------------------
  00-0-000 |   ◦ ◦ ◦  01-0-111   |   ◦ - ◦  00-0-101   |   - ◦ ◦  01-1-010   |   
  00-0-001 |   ◦ ◦ ◦  01-0-110   |   ◦ - ◦  00-0-100   |   - ◦ ◦  01-1-011   |   
  00-0-010 |   ◦ ◦ +  10-1-000   |   ◦ ◦ +  00-0-111   |   - ◦ +  10-0-101   |   
  00-0-011 |   ◦ ◦ +  10-1-001   |   ◦ ◦ +  00-0-110   |   - ◦ +  10-0-100   |   
  00-0-100 |   ◦ + ◦  01-1-110   |   ◦ + ◦  00-0-001   |   - + ◦  01-0-011   |   
  00-0-101 |   ◦ + ◦  01-1-111   |   ◦ + ◦  00-0-000   |   - + ◦  01-0-010   |   
  00-0-110 |   ◦ ◦ ◦  10-0-001   |   ◦ ◦ -  00-0-011   |   - ◦ ◦  10-1-100   |   
  00-0-111 |   ◦ ◦ ◦  10-0-000   |   ◦ ◦ -  00-0-010   |   - ◦ ◦  10-1-101   |   
------------------------------------------------------------------------------
  00-1-000 |   - ◦ ◦  01-0-010   |

# Masks to distinguish the 3 scenarios

The masks are dependent on

- number of slices rows and columns
- encoding scheme for storing (x,y,z), e.g, the z-code

In [ ]:
#S,R,C = 15,15,15
#S,R,C = 1,1,1
S,R,C = 2,4,6

In [ ]:
# MASKS, dependent on S,R,C

# 4-dimensional index 
# - ax (0,1,2), 
# - d (0-15)
# - case (corner, edgem interior)
# - matching (z,y,x)

subs = np.fromstring ('''
0 0 0 
0 0 0
0 0 1
0 0 1
0 1 0 
0 1 0
0 0 0
0 0 0
''', sep = ' ', dtype=np.uint8).reshape (8,3)

LUT_BOUNDS = np.zeros ((3,16,3), dtype=np.uint32)
LUT_BOUNDS [0, :8  ] = subs
LUT_BOUNDS [0,  8: ] = subs
LUT_BOUNDS [1] = np.roll (LUT_BOUNDS[0], shift=1, axis=-1)
LUT_BOUNDS [2] = np.roll (LUT_BOUNDS[1], shift=1, axis=-1)

LUT_BOUNDS *= np.array ([S-1,R-1,C-1], dtype=np.uint32)

LUT_BOUNDS[0, 8:, 0] = S
LUT_BOUNDS[1, 8:, 1] = R
LUT_BOUNDS[2, 8:, 2] = C

In [ ]:
# XORs: 48 darts x 3 scenarios
LUT_XORs = np.zeros((48,3), dtype=np.uint32)
for i,(z,y,x) in enumerate (LUT_BOUNDS.reshape (48,3)):
    LUT_XORs [i] = interleave3(x,y,z)

In [ ]:
# masks

msks = np.fromstring ('''
1 1 0
1 1 0
1 0 1 
1 0 1
1 1 0
1 1 0
1 0 1 
1 0 1
''', sep = ' ', dtype=np.uint32).reshape (8,3)

LUT_MASKS = np.zeros ((3,16,3,3), dtype=np.uint32)
LUT_MASKS [0,  :8, [0,1]] = msks
LUT_MASKS [0,8:16, [0,1]] = msks
LUT_MASKS [0,:,1,[1,2]] = 0

LUT_MASKS [1] = np.roll (LUT_MASKS[0],1,2)
LUT_MASKS [2] = np.roll (LUT_MASKS[1],1,2)

In [ ]:
# TODO: check if there can be any leading ones after x*(2**10-1) etc
# ANDs: 48 darts x 3 scenarios

LUT_ANDs = np.zeros((48,3), dtype=np.uint32)
for s in 0,1,2:
    for d in range (48):
        z,y,x = LUT_MASKS.reshape (48,3,3)[d,s]
        LUT_ANDs [d,s] = interleave3(x*(2**9-1),y*(2**9-1),z*(2**8-1))

## 1 XOR and 2 AND tables summary

- XOR table depends on S,R,C
- AND tables depends on ordering of coordinates, the following for z-code

In [ ]:
for d in range (0,48):
    if not d %  8: print (110*'-')

    print (
        f'{d:2}',
        f'{dartcolor (LUT_XORs[d,0]<<6, nBits=26, return_str=True)[:-18]} ', 

        f'{dartcolor (LUT_ANDs[d,0]<<6, nBits=26, return_str=True)[:-18]} ', 
        f'{dartcolor (LUT_ANDs[d,1]<<6, nBits=26, return_str=True)[:-18]} ', 
    )

--------------------------------------------------------------------------------------------------------------
 0 00.000.000.000.000.000.000.000.000  10.110.110.110.110.110.110.110.110  00.100.100.100.100.100.100.100.100 
 1 00.000.000.000.000.000.000.000.000  10.110.110.110.110.110.110.110.110  00.100.100.100.100.100.100.100.100 
 2 00.000.000.000.000.000.001.000.001  01.101.101.101.101.101.101.101.101  00.100.100.100.100.100.100.100.100 
 3 00.000.000.000.000.000.001.000.001  01.101.101.101.101.101.101.101.101  00.100.100.100.100.100.100.100.100 
 4 00.000.000.000.000.000.000.010.010  10.110.110.110.110.110.110.110.110  00.100.100.100.100.100.100.100.100 
 5 00.000.000.000.000.000.000.010.010  10.110.110.110.110.110.110.110.110  00.100.100.100.100.100.100.100.100 
 6 00.000.000.000.000.000.000.000.000  01.101.101.101.101.101.101.101.101  00.100.100.100.100.100.100.100.100 
 7 00.000.000.000.000.000.000.000.000  01.101.101.101.101.101.101.101.101  00.100.100.100.100.100.100.100.100 
-

38 00.000.000.000.000.000.000.000.000  11.011.011.011.011.011.011.011.011  01.001.001.001.001.001.001.001.001 
39 00.000.000.000.000.000.000.000.000  11.011.011.011.011.011.011.011.011  01.001.001.001.001.001.001.001.001 
--------------------------------------------------------------------------------------------------------------
40 00.000.000.000.000.000.001.001.000  01.101.101.101.101.101.101.101.101  01.001.001.001.001.001.001.001.001 
41 00.000.000.000.000.000.001.001.000  01.101.101.101.101.101.101.101.101  01.001.001.001.001.001.001.001.001 
42 00.000.000.000.000.000.001.011.010  11.011.011.011.011.011.011.011.011  01.001.001.001.001.001.001.001.001 
43 00.000.000.000.000.000.001.011.010  11.011.011.011.011.011.011.011.011  01.001.001.001.001.001.001.001.001 
44 00.000.000.000.000.000.001.001.100  01.101.101.101.101.101.101.101.101  01.001.001.001.001.001.001.001.001 
45 00.000.000.000.000.000.001.001.100  01.101.101.101.101.101.101.101.101  01.001.001.001.001.001.001.001.001 
4

In [ ]:
def alpha_2_old (d):
    """
    Implicitly encoded alpha_2
    
    Steps
        1. get zcode encoding voxel's x,y,z
        2. get 48-index of the dart for that voxel
        3. get  3-index determining the scenario
        4. get updated based on the two above indices
        5. update, recombine
        6. return the new dart
    """
    
    zcode = d >> 6
    i48   = d & 0b111111 # TODO check if this index <= 47
    
    # i3 retrieves the index 0,1,2 for cases edge, shell, interior 
    zcodeXOR = zcode ^ LUT_XORs[i48,0]                         # 1 XOR column  is necessary for all scenarios
    i3 =   (zcodeXOR & LUT_ANDs[i48,:2]).astype(np.bool).sum() # 2 AND columns are necessary, as the the 3rd one (interior) would be zero anyway ....
#   i3 = np.count_nonzero((zcode ^ LUT_XORs[i48]) & LUT_ANDs[i48])  # ORIGINAL using 3 XOR and 3 AND columns

    dz,dy,dx = LUT_DELTA [i48,i3]
    newbits  = LUT_D     [i48,i3]
    x,y,z = deinterleave3(zcode)
    x+=dx; y+=dy; z+=dz
    
    return (interleave3(x,y,z) << 6) | newbits

In [ ]:
# combined LUT experiment

# 48 x 3 x 4 entries containing dx,dy,dz,bbbbbb

LUT_DD = np.zeros ((48,3,4), dtype=np.int8)
LUT_DD [..., :3] = LUT_DELTA [...,::-1]  #   [...,::-1] => flipping Deltas z,y,x -> x,y,z
LUT_DD [...,  3] = LUT_D

# XOR and tables shifted by 6 bits
# alternative: (d XOR x) AND a = (d AND a) XOR (x AND a), the later can be precomputed
# both following tables are 48x2
LUT_XORoAND_6 = ( LUT_XORs[:,:1] & LUT_ANDs[:,:2]) << 6
LUT_AND_6     =                    LUT_ANDs[:,:2]  << 6

In [ ]:
def alpha_2 (d):
    """
    Implicitly encoded alpha_2
    
    Steps
        1. get zcode encoding voxel's x,y,z
        2. get 48-index of the dart for that voxel
        3. get  3-index determining the scenario
        4. get updated based on the two above indices
        5. update, recombine
        6. return the new dart
    """
    i48 =   d & 0b111111                                                       # dart     index
    i3  = ((d & LUT_AND_6 [i48]) ^ LUT_XORoAND_6 [i48]).astype(np.bool).sum()  # scenario index
    
#   dx,dy,dz,bbbbbb = LUT_DD [i48,i3]  # the order of the deltas is now dx,dy,dz
    x,y,z = deinterleave3(d      >> 6) + LUT_DD [i48,i3,:3]  # deinterleave the z-code and update it by dx,dy,dz
    return (  interleave3(x,y,z) << 6) | LUT_DD [i48,i3, 3]  # interleave and append bbbbbb

## $D$: Set of darts

In [ ]:
# set of darts

D = set()

for z in range (S):
    for y in range (R):
        for x in range (C):
            for i in range (48):
                D |= {interleave3 (x,y,z) << 6 | i}

z = S
for y in range (R):
    for x in range (C):
        for i in range (16):
            D |= {interleave3 (x,y,z) << 6 | i}

y = R
for x in range (C):
    for z in range (S):
        for i in range (16):
            D |= {interleave3 (x,y,z) << 6 | 0b010000 | i}
    
x = C
for y in range (R):
    for z in range (S):
        for i in range (16):
            if (interleave3 (x,y,z) << 6 | 0b100000 | i) in D:
                print ('grrr')
            D |= {interleave3 (x,y,z) << 6 | 0b100000 | i}

### involution checks

In [ ]:
# involution checks

assert all ([alpha_0(alpha_0 (d)) == d for d in D])
assert all ([alpha_1(alpha_1 (d)) == d for d in D])
assert all ([alpha_2(alpha_2 (d)) == d for d in D])

assert all ([alpha_0(alpha_2(alpha_0(alpha_2(d)))) == d for d in D])
assert all ([alpha_0(alpha_3(alpha_0(alpha_3(d)))) == d for d in D])
assert all ([alpha_1(alpha_3(alpha_1(alpha_3(d)))) == d for d in D])

### sanity check with array-based gmaps

In [ ]:
from combinatorial.gmaps import nGmap

In [ ]:
A = np.full ((4,1+max(D)),-1)  # initialize with invalid darts (-1)

for d in D:
    A[0,d] = alpha_0(d)
    A[1,d] = alpha_1(d)
    A[2,d] = alpha_2(d)
    A[3,d] = alpha_3(d)

In [ ]:
g = nGmap.from_alpha_array (A)
assert g.is_valid
assert g.no_0_cells == (S+1)*(R+1)*(C+1)
assert g.no_1_cells == S*(R+1)*(C+1) + (S+1)*R*(C+1) + (S+1)*(R+1)*C
assert g.no_2_cells == (S+1)*R*C + (R+1)*S*C + (C+1)*S*R
assert g.no_3_cells == S*R*C + 1 # also background counted
assert g.no_ccs     == 1

In [ ]:
print (f'{S}x{R}x{C} volume resulted in', g)

2x4x6 volume resulted in 3-gMap of 3008 darts:
  # 0-cells: 105
  # 1-cells: 244
  # 2-cells: 188
  # 3-cells: 49
  # ccs    : 1



# Future 

## Dart set $D$ as bit-array

In [ ]:
# voxel example, i.e, 2*48 darts
import bitarray

ba = bitarray.bitarray (2**7)

ba[:] = 0

ba[:48] = 1
ba[64:64+48] = 1

ba

bitarray('11111111111111111111111111111111111111111111111100000000000000001111111111111111111111111111111111111111111111110000000000000000')

# Dart set $D$ implicitly

2D version uncommented

In [ ]:
# check for membership

# max_x = interleave2 (C,0) << 3
# max_y = interleave2 (0,R) << 3

# mask_x = 0xffffffff ^ 2**nBits-1 | (MASK_H << 3)
# mask_y = 0xffffffff ^ 2**nBits-1 | (MASK_V << 3)

# print (f'''\
# {mask_x:032b} ... x-mask for (at most) 32-bit integer numbers
# {mask_y:032b} ... y-mask for (at most) 32-bit integer numbers
# ''')


# def is_in (d):
#     return (
#            d & mask_x <  max_x and d & mask_y <  max_y                    # x <  C and y <  R
#         or d & mask_x <  max_x and d & mask_y == max_y and not d & 0b100  # x <  C and y == R and last 3 bits are in 0...3
#         or d & mask_x == max_x and d & mask_y <  max_y and     d & 0b100  # x == C and y <  R and last 3 bits are in 4...7
#     )

In [ ]:
# assert all (    is_in(d) for d in D)

# for d in D | {dd for dd in range (len(D), len(D)+10050)}:
#         if (d in D and not is_in(d)) or (not d in D  and is_in(d)):
#             z = d >> 3
#             x,y = deinterleave2 (z)
#             print (f'{d:4} {d:08b} {x} {y} {is_in (d)}')